In [9]:
import pandas as pd
import matplotlib.pyplot as plt
# from sklearn.feature_extraction.text import TfidfVectorizer
from malnis import show
# from nltk.tokenize import word_tokenize
from tqdm.auto import tqdm
# import scipy.sparse as sp
# from sklearn.linear_model import LogisticRegression 
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.svm import SVC
# from sklearn.decomposition import PCA, SparsePCA
# from sklearn.neural_network import MLPClassifier
# from sklearn.metrics import log_loss, PrecisionRecallDisplay, RocCurveDisplay
import numpy as np
# from sentence_transformers import SentenceTransformer
from sklearn.model_selection import train_test_split
# import torch
# from torch import nn
# import pytorch_lightning as pl
# import torch.utils.data as tud
# import seaborn as sns
# sns.set()

In [10]:
# The number of sentences in each paper goes from 59 to 4,447. I truncate to 512.

data = pd.read_pickle("../data/sentence_labels.pkl")\
.reset_index(drop = True)\
.assign(original_sentences = lambda df: df.sentences.map(len))\
.assign(
    sentences = lambda df: df.sentences.map(lambda y: y[:512]),
    relevance = lambda df: df.relevance.map(lambda y: y[:512]),    
)

show(data)

(8965, 9)


,query,document,summary,r1,r2,rl,sentences,relevance,original_sentences
0,We introduce a new language representation mod...,"KEYWORDS cascade ranking, pre-trained language...",[Our approach is mainly based on the BERT lang...,0.237885,0.065359,0.229075,"[KEYWORDS cascade ranking, pre-trained languag...","[False, False, False, False, False, False, Fal...",117
1,The dominant sequence transduction models are ...,"KEYWORDS cascade ranking, pre-trained language...",[BERT [2] is a self-supervised approach for pr...,0.238372,0.063366,0.215116,"[KEYWORDS cascade ranking, pre-trained languag...","[False, False, False, False, False, True, Fals...",117
2,Language model pretraining has led to signific...,"KEYWORDS cascade ranking, pre-trained language...","[Recently, some variants [4, 12] of BERT langu...",0.172727,0.047782,0.172727,"[KEYWORDS cascade ranking, pre-trained languag...","[False, False, False, False, False, True, Fals...",117
3,With the capability of modeling bidirectional ...,"KEYWORDS cascade ranking, pre-trained language...","[Recently, some variants [4, 12] of BERT langu...",0.237838,0.078740,0.227027,"[KEYWORDS cascade ranking, pre-trained languag...","[False, False, False, False, False, False, Tru...",117
4,Neural sequence-to-sequence models have provid...,"KEYWORDS cascade ranking, pre-trained language...",[The proposed model is based on the pointer-ge...,0.215139,0.093023,0.199203,"[KEYWORDS cascade ranking, pre-trained languag...","[False, False, False, False, False, False, Fal...",117


In [11]:
embeddings_folder = "/home/jarobyte/scratch/malnis_dataset/data/embeddings/specter/"

In [12]:
X = np.load(embeddings_folder + "X.npy")
X.shape

(8965, 512, 1536)

In [13]:
Y = np.load(embeddings_folder + "Y.npy")
Y.shape

(8965, 512)

In [14]:
(
    X_train, 
    X_devtest, 
    Y_train, 
    Y_devtest, 
    data_train, 
    data_devtest
) = train_test_split(
    X, 
    Y, 
    data, 
    random_state = 1,
    test_size = 0.2
)
print("X_train", X_train.shape)
print("X_devtest", X_devtest.shape)
print("Y_train", Y_train.shape)
print("Y_devtest", Y_devtest.shape)

X_train (7172, 512, 1536)
X_devtest (1793, 512, 1536)
Y_train (7172, 512)
Y_devtest (1793, 512)


In [15]:
(
    X_dev, 
    X_test, 
    Y_dev, 
    Y_test, 
    data_dev, 
    data_test
) = train_test_split(
    X_devtest, 
    Y_devtest, 
    data_devtest, 
    random_state = 1,
    test_size = 0.5
)
print("X_dev", X_dev.shape)
print("X_test", X_test.shape)
print("Y_dev", Y_dev.shape)
print("Y_test", Y_test.shape)

X_dev (896, 512, 1536)
X_test (897, 512, 1536)
Y_dev (896, 512)
Y_test (897, 512)


In [18]:
split_folder = "/home/jarobyte/scratch/malnis_dataset/data/"

In [19]:
to_write = [
    ("X_train", X_train),
    ("X_dev", X_dev),
    ("X_test", X_test),
    
    ("Y_train", Y_train),
    ("Y_dev", Y_dev),
    ("Y_test", Y_test),
]

for name, contents in to_write:
#     contents = contents.cpu().numpy()
    path = split_folder + name + ".npy"
    np.save(path, contents)
    print(name, "saved in", path)

X_train saved in /home/jarobyte/scratch/malnis_dataset/data/X_train.npy
X_dev saved in /home/jarobyte/scratch/malnis_dataset/data/X_dev.npy
X_test saved in /home/jarobyte/scratch/malnis_dataset/data/X_test.npy
Y_train saved in /home/jarobyte/scratch/malnis_dataset/data/Y_train.npy
Y_dev saved in /home/jarobyte/scratch/malnis_dataset/data/Y_dev.npy
Y_test saved in /home/jarobyte/scratch/malnis_dataset/data/Y_test.npy


In [20]:
to_write = [
    ("data_train", data_train),
    ("data_dev", data_dev),
    ("data_test", data_test),
]

for name, contents in to_write:
#     contents = contents.cpu().numpy()
    path = split_folder + name + ".pkl"
    contents.to_pickle(path)
    print(name, "saved in", path)

data_train saved in /home/jarobyte/scratch/malnis_dataset/data/data_train.pkl
data_dev saved in /home/jarobyte/scratch/malnis_dataset/data/data_dev.pkl
data_test saved in /home/jarobyte/scratch/malnis_dataset/data/data_test.pkl
